In [1]:
import numpy as np
import timeit as tt

from pynq import Xlnk
from pynq import Overlay

weights = np.array( [[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=np.float32)
inputs = np.array([[0.1,0.2,0.3]], dtype=np.float32)
bias = np.array([-2.2, -3.33, -4.444, -5.5555], dtype=np.float32)
dots = np.dot(inputs,weights)
b_dots = dots + bias 
outs = np.tanh(b_dots)

print ("Weights:\n", weights)
print ("Inputs:\n", inputs)
print ("Bias: \n", bias)
print ("Dot Product:\n", dots)
print ("Biased Products:\n", b_dots)
print ("Output:\n", outs)

overlay = Overlay('test.bit')
dma = overlay.axi_dma_0
xlnk = Xlnk()

input_buffer = xlnk.cma_array(shape=(len(inputs[0]),), dtype=np.float32)
np.copyto(input_buffer,  inputs[0])
output_buffer = xlnk.cma_array(shape=(len(outs[0]),), dtype=np.float32)

dma.sendchannel.transfer(input_buffer)    
dma.recvchannel.transfer(output_buffer)

dma.sendchannel.wait()
dma.recvchannel.wait()

output_buffer = output_buffer.reshape(1, len(output_buffer))

print ("FPGA Output:\n", output_buffer)

Weights:
 [[  1.   2.   3.   4.]
 [  5.   6.   7.   8.]
 [  9.  10.  11.  12.]]
Inputs:
 [[ 0.1         0.2         0.30000001]]
Bias: 
 [-2.20000005 -3.32999992 -4.44399977 -5.55550003]
Dot Product:
 [[ 3.80000019  4.4000001   5.          5.60000038]]
Biased Products:
 [[ 1.60000014  1.07000017  0.55600023  0.04450035]]
Output:
 [[ 0.92166859  0.78946131  0.50500375  0.044471  ]]
FPGA Output:
 [[ 0.92072231  0.79824275  0.48633602  0.03123983]]
